In [2]:
from email import header
import requests
import json
from lxpy import copy_headers_dict
import pandas as pd
import os 
from os import listdir
from os.path import isfile, join
# import tkinter as tk
# import tkinter.ttk as ttk
import datetime
from bs4 import BeautifulSoup as bs
import aiohttp
import time
import asyncio
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
from sqlalchemy import create_engine
from tqdm.notebook import tqdm
import aiohttp
import asyncio
from bs4 import BeautifulSoup as bs
import nest_asyncio

In [4]:
set_headers=copy_headers_dict('''
accept:text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
accept-encoding:gzip, deflate, br, zstd
accept-language:en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7
cache-control:no-cache
pragma:no-cache
priority:u=0, i
sec-ch-ua:"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"
sec-ch-ua-mobile:?0
sec-ch-ua-platform:"macOS"
sec-fetch-dest:document
sec-fetch-mode:navigate
sec-fetch-site:none
sec-fetch-user:?1
upgrade-insecure-requests:1
user-agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0
''')
url="https://accessnc.nccommerce.com/BusinessSearch/Company/list"
trash='''file:///'''
counties = [
    "Alexander", "Alleghany", "Ashe", "Avery", "Buncombe", "Burke",
    "Caldwell", "Catawba", "Clay", "Cleveland", "Gaston", "Haywood",
    "Henderson", "Jackson", "Lincoln", "Macon", "Madison", "McDowell",
    "Mecklenburg", "Mitchell", "Polk", "Rutherford", "Swain",
    "Transylvania", "Watauga", "Wilkes", "Yancey"
]


## Example Access

In [9]:
data = {
    "regionCategory": "County",  # Example: "County" or "Region"
    "region": "Cleveland",  # Example: "Wake" for Wake County
    "IndustryGroup2": "6221",  # NAICS 4-digit code, e.g., "1124" for Sheep and Goat Farming
    # "CompanyName": "",  # Optional: Add a company name or keyword
    # "stype": "c",  # Match type for CompanyName: 'c' (contains) or 's' (starts with)
    # "STFIPS": "37",  # State FIPS code for North Carolina
}

# Send POST request
response = requests.post(url, data=data)

# Check the response
if response.status_code == 200:
    print("Search successful!")
    print(response.text)  # or parse the response as needed
else:
    print(f"Search failed with status code {response.status_code}")


Search successful!

<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Business Search</title>
    <link href="/BusinessSearch/Content/css?v=MDbdFKJHBa_ctS5x4He1bMV0_RjRq8jpcIAvPpKiN6U1" rel="stylesheet"/>

    <script src="/BusinessSearch/bundles/modernizr?v=wBEWDufH_8Md-Pbioxomt90vm6tJN2Pyy9u9zHtWsPo1"></script>


</head>
<body>
    <div class="navbar navbar-inverse navbar-fixed-top">
        <a class="navbar-brand" href="https://d4.nccommerce.com">D4</a>
        
    </div>
    <div class="container body-content">
        


<link href="/BusinessSearch/Content/PagedList.css" rel="stylesheet" type="text/css" />



<h3>Business Search Results</h3>
<p>
    Business Search Results  657977 matched records   
     
</p>
<table class="table">
    <tr>
        <th>
            <a href="/BusinessSearch/Company/sort?sortOrder=company">Company</a>
        </th>
        <th>
            <a href="/Busin

In [10]:
html_parse=bs(response.text,'html.parser')
for link in html_parse.find_all("a", string=lambda text: text == "Detail"):
    print(link)

<a href="/BusinessSearch/Company/Detail/762283043">Detail</a>
<a href="/BusinessSearch/Company/Detail/734095575">Detail</a>
<a href="/BusinessSearch/Company/Detail/745908587">Detail</a>
<a href="/BusinessSearch/Company/Detail/723354960">Detail</a>
<a href="/BusinessSearch/Company/Detail/740424324">Detail</a>
<a href="/BusinessSearch/Company/Detail/401678984">Detail</a>
<a href="/BusinessSearch/Company/Detail/220228084">Detail</a>
<a href="/BusinessSearch/Company/Detail/705185549">Detail</a>
<a href="/BusinessSearch/Company/Detail/725665057">Detail</a>
<a href="/BusinessSearch/Company/Detail/717072561">Detail</a>
<a href="/BusinessSearch/Company/Detail/435029225">Detail</a>
<a href="/BusinessSearch/Company/Detail/742698417">Detail</a>
<a href="/BusinessSearch/Company/Detail/722866552">Detail</a>
<a href="/BusinessSearch/Company/Detail/742138490">Detail</a>
<a href="/BusinessSearch/Company/Detail/722799086">Detail</a>
<a href="/BusinessSearch/Company/Detail/747310914">Detail</a>
<a href=

## All Links

In [7]:
# Read NAICS codes from the file
with open("qnaics_codes.txt", 'r') as f:
    codes = f.readlines()

# Clean and prepare data
codes = list(map(lambda x: x.strip(), codes))
datalist = []
for county in counties:
    for code in codes:
        datalist.append({
            "regionCategory": "County",  # Example: "County" or "Region"
            "region": county,  # Example: "Wake" for Wake County
            "IndustryGroup2": code,  # NAICS 4-digit code, e.g., "1124" for Sheep and Goat Farming
            # "CompanyName": "",  # Optional: Add a company name or keyword
            # "stype": "c",  # Match type for CompanyName: 'c' (contains) or 's' (starts with)
            # "STFIPS": "37",  # State FIPS code for North Carolina
        })

# URL for form submission
url = "https://accessnc.nccommerce.com/BusinessSearch/Company/list"


In [12]:
len(datalist)

16794

In [14]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup as bs
from tqdm.notebook import tqdm
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Load NAICS codes and counties
with open("qnaics_codes.txt", 'r') as f:
    codes = [line.strip() for line in f]

# List of counties
counties = [
    "Alexander", "Alleghany", "Ashe", "Avery", "Buncombe", "Burke",
    "Caldwell", "Catawba", "Clay", "Cleveland", "Gaston", "Haywood",
    "Henderson", "Jackson", "Lincoln", "Macon", "Madison", "McDowell",
    "Mecklenburg", "Mitchell", "Polk", "Rutherford", "Swain",
    "Transylvania", "Watauga", "Wilkes", "Yancey"
]

# Prepare datalist
datalist = []
for county in counties:
    for code in codes:
        datalist.append({
            "regionCategory": "County",
            "region": county,
            "IndustryGroup2": code,
        })

# URL for form submission
url = "https://accessnc.nccommerce.com/BusinessSearch/Company/list"

# Function to split datalist into batches
def split_into_batches(data, num_batches):
    batch_size = len(data) // num_batches
    return [data[i * batch_size:(i + 1) * batch_size] for i in range(num_batches)]

# Split datalist into 6 batches
batches = split_into_batches(datalist, 6)

# Asynchronous function to fetch and process data
async def fetch_data(session, data):
    try:
        async with session.post(url, data=data) as response:
            await asyncio.sleep(0.05)  # Rate limiting
            if response.status == 200:
                html_parse = bs(await response.text(), 'html.parser')
                links = []
                for link in html_parse.find_all("a", string=lambda text: text == "Detail"):
                    local_link = link.get("href")
                    links.append(local_link)
                    with open("all_links.txt", "a") as f:
                        f.write(f"{local_link}\n")
                return links
            else:
                print(f"Failed for data: {data} with status {response.status}")
    except Exception as e:
        print(f"Error processing data: {data} - {e}")

# Main asynchronous function to process a batch
async def process_batch(batch):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_data(session, data) for data in batch]
        results = []
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing Batch"):
            result = await f
            results.append(result)
        return results

# Sequentially process batches with rest between each batch
async def main():
    for i, batch in enumerate(batches):
        print(f"Processing Batch {i + 1}/{len(batches)}")
        await process_batch(batch)
        if i < len(batches) - 1:  # Rest only if there are more batches left
            print("Resting for 10 seconds...")
            await asyncio.sleep(10)
    print("All batches processed!")

# Run the asynchronous main function
await main()


Processing Batch 1/6


Processing Batch:   0%|          | 0/2799 [00:00<?, ?it/s]

Resting for 10 seconds...
Processing Batch 2/6


Processing Batch:   0%|          | 0/2799 [00:00<?, ?it/s]

Resting for 10 seconds...
Processing Batch 3/6


...

Error processing data: {'regionCategory': 'County', 'region': 'McDowell', 'IndustryGroup2': '3161'} - 
Error processing data: {'regionCategory': 'County', 'region': 'Lincoln', 'IndustryGroup2': '2381'} - 
Error processing data: {'regionCategory': 'County', 'region': 'McDowell', 'IndustryGroup2': '3159'} - 
Error processing data: {'regionCategory': 'County', 'region': 'McDowell', 'IndustryGroup2': '3122'} - 
Error processing data: {'regionCategory': 'County', 'region': 'McDowell', 'IndustryGroup2': '3151'} - 
Error processing data: {'regionCategory': 'County', 'region': 'McDowell', 'IndustryGroup2': '3141'} - 
Error processing data: {'regionCategory': 'County', 'region': 'Lincoln', 'IndustryGroup2': '3118'} - 
Error processing data: {'regionCategory': 'County', 'region': 'Lincoln', 'IndustryGroup2': '3141'} - 
Error processing data: {'regionCategory': 'County', 'region': 'McDowell', 'IndustryGroup2': '3133'} - 
Error processing data: {'regionCategory': 'County', 'region': 'McDowell', 'I

Processing Batch:   0%|          | 0/2799 [00:00<?, ?it/s]

Resting for 10 seconds...
Processing Batch 6/6


Processing Batch:   0%|          | 0/2799 [00:00<?, ?it/s]

All batches processed!


In [ ]:
all_links = []
base_url = "https://accessnc.nccommerce.com"
with open("all_links.txt","r") as f:
    all_links = f.readlines()
all_links_full = list(set(list(map(lambda x: base_url + x, all_links))))
with open("all_links_counties_full.txt","w") as f:
    f.writelines(all_links_full)

1669800

In [32]:
alf=(list(map(lambda x: base_url + x.strip(),all_links)))

In [ ]:
from collections import Counter


['https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/762283043',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/734095575',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/745908587',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/723354960',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/740424324',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/401678984',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/220228084',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/705185549',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/725665057',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/717072561',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/435029225',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/742698417',
 'https://accessnc.nccommerce.com/BusinessSearch/Company/Detail/722866552',
 'https://ac